#### The content in this file is based on: https://github.com/pescap/HPOMax/blob/main/HPO-PINNs/Dirichlet/HPO-2D.ipynb

In [ ]:
import skopt
import seaborn as sns
from model_gp import fitness
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
from matplotlib.colors import LogNorm, Normalize
import seaborn as sns

In [ ]:
# Cost as function of depth and length
def formula(N , L):
    return N * (2+ 1) + (L-2) * N * (N + 1) + (N+1) 


def parse_results(name):
    search_result = skopt.load(name + '.pkl')       # load search_result
    n_calls = search_result.func_vals.shape[0]      # get n_calls
    test = []
    metric = []
    train = []
    texec = []

    # append results from all configs
    for i in range(n_calls):
        temp_name = name + '-'  + str(i) 
        train.append(skopt.load(temp_name + 'train.pkl'))
        test.append(skopt.load(temp_name + 'test.pkl')) 
        metric.append(skopt.load(temp_name + 'metric.pkl'))
        texec.append(skopt.load(temp_name + 'texec.pkl'))      

    # allocate
    min_train = np.zeros(n_calls)
    min_test = np.zeros(n_calls)
    min_metric = np.zeros(n_calls)

    # Find mest values for each config
    for i in range(n_calls):
        imin = test[i].argmin()
        min_train[i] = train[i][imin]
        min_test[i] = test[i][imin]
        min_metric[i] = metric[i][imin]

    # create dataframe of best values
    df1 = pd.DataFrame(np.column_stack([min_train, min_test, min_metric, texec]), columns = ['train', 'test', 'metric', 'texec'])
    print(df1)
    # Remove nan in metric
    max_train = df1[~np.isnan(df1.train)].max()
    max_test =  df1[~np.isnan(df1.test)].max()
    max_metric = df1[~np.isnan(df1.metric)].max()
    
    # Set nan values to max
    df1[np.isnan(df1.train)] = max_train
    df1[np.isnan(df1.test)] = max_test
    df1[np.isnan(df1.metric)] = max_metric


    # Create dataframe of all configs
    df0 = pd.DataFrame(search_result.x_iters, columns=['lr', 'd', 'w', 'a']) 
    df0['f'] = search_result.func_vals.astype(float)        # function values for each iteration
    df0['global_index'] = np.arange(df0.shape[0])           # Configuration number

    # Information: learning rate, depth, width, activation
    df0['information'] = '[' +df0.lr.map(lambda x : "{:.2e}".format(x)).astype(str)+ ',' + df0.d.astype(str) + ','+ df0.w.astype(str)+ ',' + df0.a.astype(str) + ']'
    df0['cost'] = formula(df0.w, df0.d + 1)  
    
    # Concatenate data frames
    df2 = pd.concat([df0, df1], axis = 1)
    
    # allocate
    conv = np.zeros(n_calls)
    conv_metric = np.zeros(n_calls)
    
    # Conv, conv_metric columns
    for i in range(n_calls):
        conv[i] = df2.f.values[:i + 1].min()
        conv_metric[i] = df2.metric.values[:i + 1].min()
    df2['conv'] = conv
    df2['conv_metric'] = conv_metric
    
    df2.loc[df2.f == 100000, 'f'] = df2.f[df2.f != 100000].max()
    
    # sort by function values
    df = df2.sort_values(['f'], ascending = [True])

    # Convert types and format
    df['cost'] = df['cost'].astype(float)
    df[['global_index','f']] = df[['global_index','f']].astype(float)
    df['f'] = df['f'].astype(float)
    df['f_format'] = df.f.map(lambda x : "{:.2e}".format(x)).astype(str)
    df['cost_format'] = df.cost.map(lambda x : "{:.2e}".format(x)).astype(str)
    
    return df, train, test, metric





In [ ]:
name = 'results/burgersgp'

n_calls = 30
df, trainF, testF, metricF = parse_results(name)

# new columns
df['local_index'] = (np.arange(n_calls) + 1).astype(int)    # index here
df['log10f'] = np.log10(df.f)                               # log10 transform of f
df['log10metric'] = np.log10(df.metric)                     # log10 transform of metric
df['log10train'] = np.log10(df.train)                       # log10 transform of train
df['log10test'] = np.log10(df.test)                         # log10 transform of test
df['log10conv'] = np.log10(df.conv)                         # log10 transform of conv
df['log10conv_metric'] = np.log10(df.conv_metric)           # log10 transform of conv_metric

search_result = skopt.load(name + '.pkl')                   # load search_result

df

In [ ]:
melt1 = df.melt(id_vars =['local_index'], value_vars =  ['log10train', 'log10test'])
melt2 = df.melt(id_vars =['global_index'], value_vars =  ['log10conv','log10conv_metric'])
sns.set_style("ticks",{'axes.grid' : True})

# 1 row with 2 plots
fig, (ax1, ax2) = plt.subplots(1,2 , sharey=True, figsize= (12, 5))

g = sns.pointplot(data = melt1, 
                 x = 'local_index',                                   
                 palette = ["blue","red"], # colors
                 y = 'value',
                 scale = .8,#, s = 80, lw = 2,
                 hue='variable',
                 ax = ax1) # plug into ax1
                 

# title
ax1.set_title(r'Sorted by PDE loss', fontsize=16)

# labels
ax1.set_ylabel(r'$\log_{10}($loss$[\lambda])$', fontsize=13)
ax1.set_xlabel(r'Ranking', fontsize=16)

ax1.set_xlim(0,30)                 # axis length
xticks = np.arange(0,30,3)    # ticks on x-axis
ax1.grid()                          # grid
labels = ax1.get_xticklabels()      # get x labels

ax1.set_xticks(xticks)
a = (xticks + 1).astype(str)
b = ['#' + s for s in list(a)]
ax1.set_xticklabels(b, fontsize= 14)
ax1.xaxis.grid(color='gray', linestyle='dashed')
ax1.yaxis.grid(color='gray', linestyle='dashed')

handles, labels = ax1.get_legend_handles_labels() 

labels = [r'PDE + $\nabla$PDE loss', 'PDE loss',]
legend = ax1.legend(loc = 'lower right', handles = handles, labels = labels, title = 'Plot:', fontsize=14)  
legend.get_title().set_fontsize('14')




# AX2

g = sns.pointplot(data = melt2, x = 'global_index', 
                 palette =["red"] ,
                 y = 'value',
                 scale = .8,#, s = 80, lw = 2,
                 hue='variable',
                 ax = ax2)
ax2.set_xlabel(r'Iteration $m$', fontsize=16)


ax2.set_title(r'Convergence', fontsize=16)
ax2.set_ylabel(r'$\log_{10}($loss$^\cdot[\lambda^+_m])$', fontsize=14)
ax2.grid()

xticks = np.arange(1,30,3)

labels = ax2.get_xticklabels() # get x labels
labelsy = ax1.get_yticklabels() # get x labels

ax1.tick_params(axis='both', which='major', labelsize=13)

ax2.set_xticks(xticks)
ax2.set_xticklabels((xticks).astype(str), fontsize=14)

ax2.xaxis.grid(color='gray', linestyle='dashed')
ax2.yaxis.grid(color='gray', linestyle='dashed')
#title = ax.get_title()


handles, labels = ax2.get_legend_handles_labels() 

labels = ['PDE loss']
legend = ax2.legend(loc = 'upper right', handles = handles, labels = labels, title = 'Plot:',fontsize =14)  
legend.get_title().set_fontsize('14')


plt.subplots_adjust(wspace=0.075, hspace=0.075)

plt.savefig('plots/burgers_hypopt_result.pdf')

# Accuracy

Restore best model

In [ ]:
import deepxde as dde
if dde.backend.backend_name == "pytorch":
    sin = dde.backend.pytorch.sin
    exp = dde.backend.pytorch.exp
else:
    from deepxde.backend import tf
    sin = tf.sin
    exp = tf.exp

# IC + BC
def output_transform(x, y):
    x_in = x[:, 0:1]
    t_in = x[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - exp(-t_in)) * y - sin(np.pi * x_in)

# gPINN
def pde(x, y):
    dy_x = dde.grad.jacobian(y, x, j=0)
    dy_t = dde.grad.jacobian(y, x, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)

    dy_tx = dde.grad.hessian(y, x, i=0, j=1)
    dy_xxx = dde.grad.jacobian(dy_xx, x, j=0)

    dy_tt = dde.grad.hessian(y, x, i=1, j=1)
    dy_xxt = dde.grad.jacobian(dy_xx, x, j=1)
    return [
        dy_t + y * dy_x - 0.01 / np.pi * dy_xx,
        dy_tx + (dy_x * dy_x + y * dy_xx) - 0.01 / np.pi * dy_xxx,
        dy_tt + dy_t * dy_x + y * dy_tx - 0.01 / np.pi * dy_xxt,
    ]


geom = dde.geometry.Interval(-1, 1)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Data
data = dde.data.TimePDE(
    geomtime, 
    pde, 
    [], 
    num_domain=config.num_domain, 
)

### must be filled according to best model
net = dde.maps.FNN(
    [3] + [--fill--] * --fil-- + [3],
    "tanh",
    "Glorot normal",


model = dde.Model(data, net) 
model.compile("adam", lr=config.learning_rate)

### best model must be specified
model.restore(r"models/")

In [ ]:
def gen_testdata():
    data = np.load("Burgers.npz")
    t, x, exact = data["t"], data["x"], data["usol"].T
    xx, tt = np.meshgrid(x, t)
    X = np.vstack((np.ravel(xx), np.ravel(tt))).T
    y = exact.flatten()[:, None]
    return X, y

In [ ]:
x_true, y_true = gen_testdata()
y_pred = model.predict(x_true)
err = np.abs(y_pred - y_true)
max_err = err.max()
avg_err = err.mean()
max_err, avg_err

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.tricontourf(x_true[:,0], x_true[:,1], err.reshape(-1), levels=400)
plt.colorbar(label="absolute error")
plt.scatter(data.anchors[:,0],data.anchors[:,1], s=2, label="anchors", color="green")
plt.xlabel("x", fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.ylabel("t", fontsize=16)
plt.xlim(-1,1)
plt.ylim(0,1)
plt.title("Prediction error and RAR points")
plt.legend(fontsize=12)
plt.tight_layout(w_pad=0)
plt.savefig("burgers_error_anchors.png",
            facecolor="white",
            pad_inches=0)